# 0. Import Packages


In [62]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, text


# 1. Define Preprocessors


In [110]:
def get_data_from_csv(location, drop_col = None):
    if drop_col:
        df = pd.read_csv(location)
        df = df.drop([drop_col], axis = 1)
    else:
        df = pd.read_csv(location)
    return df



def execute_select_query(engine, query):
    """
    주어진 SQLAlchemy 엔진과 SQL 쿼리를 사용하여 쿼리를 실행하고 결과를 데이터프레임으로 반환하는 함수.
    :param engine: SQLAlchemy 엔진 객체
    :param query: 실행할 SQL 쿼리 문자열
    :return: 쿼리 결과를 포함하는 pandas DataFrame
    """
    with engine.connect() as connection:
        result = connection.execute(text(query))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df


def execute_query(engine, query):
    """
    주어진 SQLAlchemy 엔진과 SQL 쿼리를 사용하여 쿼리를 실행하고 결과를 데이터프레임으로 반환하는 함수.
    :param engine: SQLAlchemy 엔진 객체
    :param query: 실행할 SQL 쿼리 문자열
    :return: 쿼리 결과를 포함하는 pandas DataFrame
    """
    with engine.connect() as connection:
        connection.execute(text(query))


def execute_insert_query(engine, df, table_name):
    """
    주어진 SQLAlchemy 엔진과 데이터프레임을 사용하여 테이블에 데이터를 삽입하는 함수.
    :param engine: SQLAlchemy 엔진 객체
    :param df: 삽입할 데이터프레임
    :param table_name: 데이터가 삽입될 테이블 이름
    """
    with engine.connect() as connection:
        # 데이터프레임의 각 행을 사전으로 변환
        df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

        print(f"{df.shape[0]} rows has been successfully inserted into the '{table_name}' table.")




# 2. DB Connection


In [103]:
# AWS RDS MySQL 데이터베이스 접속 정보
db_host = 'mydnovadb.c7u0am6wacri.ap-northeast-2.rds.amazonaws.com'
db_name = 'YTDB'
db_user = 'mydnovadb'
db_password = 'dnova998877'
db_port = '3306'

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# 3. Basic Query


In [111]:
"""
query = 'TRUNCATE SHORTS_INFO'
x = execute_query(engine,query)
"""

query = 'SELECT * FROM SHORTS_INFO'
x = execute_select_query(engine,query)
x

,chanel_name,yt_url,subscribers,check_date,released_date,shorts_name,description,likes,views,short_url,thumbnail_link


# 4. Insert Query


In [112]:
df_to_insert = get_data_from_csv('successful_Data/data-01-07-2024.csv','is_posted_in_date_range')
table_name = 'SHORTS_INFO'
execute_insert_query(engine,df_to_insert,table_name)

182 rows has been successfully inserted into the 'SHORTS_INFO' table.
